In [23]:
import pandas as pd
from collections import Counter

def split_and_clean_course_code(l):
    if type(l) != str:
        return []
    tmp = l.replace('+',',').replace('/',',').split(',')
    return [c.strip() for c in tmp]

In [24]:
Projections = {
                'Old': ['StudentsAudit_Fatna.xlsm','STUD_ID','Manual - AUDIT-After priority change'],
                'New': ['StudentsAudit.xlsx','ID','Audit_Courses_to_add'],
                # 'New': ['StudentsAudit_old.xlsx','ID','Audit_Courses_to_add']
}

In [25]:
old = pd.read_excel(Projections['Old'][0])[Projections['Old'][1:]]
new = pd.read_excel(Projections['New'][0])[Projections['New'][1:]]
old.columns = ['ID','old']
new.columns = ['ID','new']
old['oID'] = old.ID
new['nID'] = new.ID
df = pd.merge(old, new, on='ID', how='outer')

count = len(df)
mask1 = df.oID.isnull() & df.nID.notnull()
mask2 = df.oID.notnull() & df.nID.isnull()
count1 = len(df[mask1])
count2 = len(df[mask2])
if count1:
    print(f'found {count1} in new but missing from old ({count1/count*100:.2f}%). Excluding the following records:')
    print(df[mask1])
    df.drop(df[mask1].index, inplace=True)
if count2:
    print(f'\nfound {count2} in old but missing from new ({count2/count*100:.2f}%). Excluding the following records:')
    print(df[mask2])
    df.drop(df[mask2].index, inplace=True)
df.drop(['oID','nID'],inplace=True,axis=1)

print(f'\n{len(df)} records left after cleaning.\n')

found 827 in new but missing from old (45.84%). Excluding the following records:
             ID  old  oID                             new        nID
0     201380789  NaN  NaN                             NaN  201380789
5     201603065  NaN  NaN  GEN185, FYE125, IMT340, SWE346  201603065
6     201603659  NaN  NaN                             NaN  201603659
10    201618308  NaN  NaN                             NaN  201618308
14    201700140  NaN  NaN                             NaN  201700140
...         ...  ...  ...                             ...        ...
1798  M80009359  NaN  NaN                             NaN  M80009359
1800  M80009361  NaN  NaN                             NaN  M80009361
1801  M80009362  NaN  NaN                             NaN  M80009362
1802  M80009363  NaN  NaN                             NaN  M80009363
1803  M80009364  NaN  NaN                             NaN  M80009364

[827 rows x 5 columns]

found 8 in old but missing from new (0.44%). Excluding the followi

/var/folders/1h/9dy29_294jn1f5_6gv0cd6km0000gq/T/ipykernel_29555/3084789119.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask2])
/var/folders/1h/9dy29_294jn1f5_6gv0cd6km0000gq/T/ipykernel_29555/3084789119.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop(df[mask2].index, inplace=True)


In [26]:
df.old = df.old.apply(split_and_clean_course_code)
df.new = df.new.apply(split_and_clean_course_code)
df['Delta_old'] = pd.Series(dtype='object')
df['Delta_new'] = pd.Series(dtype='object')
df['Match'] = pd.Series(dtype='float')

for i, r in df.iterrows():
    o = Counter(r.old)
    n = Counter(r.new)
    Delta_o_n = list((o-n).elements())
    Delta_n_o = list((n-o).elements())
    df.at[i,'Delta_old'] = Delta_o_n
    df.at[i,'Delta_new'] = Delta_n_o
    if len(r.old)+len(r.new) == 0:
        df.at[i,'Match'] = 1
    else:
        df.at[i,'Match'] = 1 - (len(Delta_o_n)+len(Delta_n_o)) / (len(r.old)+len(r.new))

df.to_excel('Compare_Projections.xlsx')
# display(df)

print(f'\ncompared {len(df)} students, with similarity ratio: {df.Match.sum()/len(df)*100:.2f}%')
print(f'\nHere are the {len(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])} differences:')
display(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])


compared 969 students, with similarity ratio: 84.33%

Here are the 323 differences:


,ID,old,new,Delta_old,Delta_new,Match
1,201402842,"[CIT466, NET351, SEC430, SWE320, P_Elective]",[],"[CIT466, NET351, SEC430, SWE320, P_Elective]",[],0.000000
3,201514581,"[SWE320, SWE321, CIT460, CIT461, CIT466, SEC435]","[CIT466, SWE320, SWE321, CIT460, CIT461, SEC33...",[SEC435],"[SEC330, SEC331]",0.769231
9,201618287,[],[FYE125],[],[FYE125],0.000000
11,201619114,"[GEN130(FYE145), GEN135(FYE120), GEN255(FYE125...","[SEC335, SEC336, NET351, NET352, CIT466]","[GEN130(FYE145), GEN135(FYE120), GEN255(FYE125...","[NET351, NET352, CIT466]",0.363636
24,201708865,"[CIT315, SWE320, SWE321, INS377, NET351, NET352]","[CIT315, SEC335, SEC336, NET351, NET352, SWE32...",[INS377],"[SEC335, SEC336]",0.769231
...,...,...,...,...,...,...
1723,M80009116,"[ZU Elective, CIT285, CIT365, NET256, NET257, ...","[ENG140, SWE225, NET256, NET257, ARA130, GroupA]","[ZU Elective, CIT285, CIT365]","[ENG140, ARA130, GroupA]",0.500000
1724,M80009120,"[ZU elective, CIT285, CIT365, NET256, NET257, ...","[ENG140, SWE225, CIT365, ARA130, GroupA]","[ZU elective, CIT285, NET256, NET257]","[ENG140, ARA130, GroupA]",0.363636
1725,M80009121,"[ZU elective, CIT285, CIT365, NET256, NET257, ...","[ENG140, SWE225, NET256, NET257, ARA130, GroupA]","[ZU elective, CIT285, CIT365]","[ENG140, ARA130, GroupA]",0.500000
1726,M80009122,"[ZU elective, CIT285, CIT365, NET256, NET257, ...","[ENG140, SWE225, NET256, NET257, ARA130, GroupA]","[ZU elective, CIT285, CIT365]","[ENG140, ARA130, GroupA]",0.500000
